# Week 6 Homework

***Due (pushed to your GitHub branch) on 10/25 by 11:59 pm***

## STAR two-pass alignment

Build the STAR SA:

In [4]:
%mkdir alignment/star_elegans/

!STAR \
--runThreadN 32 \
--runMode genomeGenerate \
--genomeDir ../2_genome_exploration/c_elegans_genome/star \
--genomeFastaFiles ../2_genome_exploration/c_elegans_genome/c_elegans_genome.fa \
--sjdbGTFfile ../2_genome_exploration/c_elegans_genome/c_elegans_annotations.gtf \
--sjdbOverhang 130 \
--genomeSAindexNbases 13

mkdir: cannot create directory ‘alignment/star_elegans/’: File exists
	STAR --runThreadN 32 --runMode genomeGenerate --genomeDir ../2_genome_exploration/c_elegans_genome/star --genomeFastaFiles ../2_genome_exploration/c_elegans_genome/c_elegans_genome.fa --sjdbGTFfile ../2_genome_exploration/c_elegans_genome/c_elegans_annotations.gtf --sjdbOverhang 130 --genomeSAindexNbases 13
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Nov 19 15:27:43 ..... started STAR run
Nov 19 15:27:43 ... starting to generate Genome files
Nov 19 15:27:44 ..... processing annotations GTF
!!!!! WARNING: while processing sjdbGTFfile=../2_genome_exploration/c_elegans_genome/c_elegans_annotations.gtf, line:
IV	WormBase	exon	17186906	17187473	.	+	.	gene_id "WBGene00016877"; gene_version "1"; transcript_id "C52D10.3b.1"; exon_number "5"; gene_source "WormBase"; gene_biotype "protein_coding"; transcript_source "WormBase"; transcript_biotype "protein_codi

Use STAR and ***all*** of your FASTQs for a first-pass alignment. Be sure to update the `sjdbOverhang` parameter to a value that is appropriate for your dataset. Rename the log file so that you can compare the logs from the first and second alignments:

In [2]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/c_elegans_genome/star \
--readFilesManifest manifest_elegans.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix alignment/star_elegans/

!cp alignment/star_elegans/Log.final.out alignment/star_elegans/first-pass.final.out

	STAR --runThreadN 32 --runMode alignReads --genomeDir ../2_genome_exploration/c_elegans_genome/star --readFilesManifest manifest_elegans.tsv --readFilesCommand zcat --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outFileNamePrefix alignment/star_elegans/
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Nov 18 20:43:02 ..... started STAR run
Nov 18 20:43:45 ..... loading genome
Nov 18 20:43:46 ..... started mapping
^C
cp: cannot stat 'alignment/star_elegans/Log.final.out': No such file or directory


Use STAR, all of your FASTQs, and `SJ.out.tab` for the second-pass alignment. You will likely need to make your own `manifest.tsv`:

In [ ]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/c_elegans_genome/star \
--readFilesManifest manifest_elegans.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix alignment/star_elegans/ \
--sjdbFileChrStartEnd alignment/star_elegans/SJ.out.tab

!cp alignment/star_elegans/Log.final.out alignment/star_elegans/second-pass.final.out

Update this block to briefly describe the differences between the logs for the first and second alignments.

## HISAT alignment

(***Note***: you may need Dr. Wheeler to build the HISAT index of your genome. Ask him for assistance when you get to this part of the assignment.)

Use a Bash `while` loop to align all of your FASTQs and save the output files using the same names as the input FASTQ:

In [ ]:
%mkdir alignment/hisat_elegans/

!while read -r line; do \
    fq=$(echo $line | awk '{print $1}'); \
    bn=$(basename "$fq" .fastq.gz); \
    rg=$(echo $line | awk '{print $3}' | sed 's/ID://' ); \
    echo "Aligning $fq with $rg"; \
    hisat2 /data/users/willetse0745/BIOL343/2_genome_exploration/c_elegans_genome/hisat_elegans/genome -p 16 -U $fq --rg-id $rg --rg SM:$rg --summary-file alignment/hisat_elegans/$bn.log --new-summary > alignment/hisat_elegans/$bn.sam; \
    done < manifest_elegans.tsv

## Snakemake Pipeline

In contrast to last week (FASTQ files), there aren't as many decisions that need to be made during alignment, so using a notebook to control and report code isn't as necessary. As you saw in in [6_alignment](6_alignment.ipynb), file and directory management can get complicated during alignment. This is a case where Snakemake scripting shines.

1. Copy your `Snakefile` and `config.yaml` from `5_fastq` to `6_alignment`. 
2. In the `Snakefile`, add the following rules:  
    a. `fetch_genome` - using a variation from the code from `2_genome_exploration`, download your reference genome and annotations.  
    b. `star_sa` - use `STAR` to produce the suffix array of the reference genome.  
    c. `star_align_first` - use `STAR` to perform first-pass alignment with all FASTQ files.  
    d. `star_align_second` - use `STAR` to perform second-pass alignment with all FASTQ files.  
    e. `hisat_align` - use `HISAT` to align all FASTQ files.  
    f. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.